In [1]:
import os,sys
import ROOT
import h5py
import numpy as np

Welcome to JupyROOT 6.28/04


In [2]:
rfilename = "new_magnify2-28052-20833.root"

In [3]:
rfile = ROOT.TFile(rfilename,"read")
rfile.ls()

TFile**		new_magnify2-28052-20833.root	
 TFile*		new_magnify2-28052-20833.root	
  KEY: TH2I	h_orig;1	h_orig
  KEY: TH2I	hu_orig;1	hu_orig
  KEY: TH2I	hv_orig;1	hv_orig
  KEY: TH2I	hw_orig;1	hw_orig
  KEY: TH1I	h_baseline;1	h_baseline
  KEY: TH1I	hu_baseline;1	hu_baseline
  KEY: TH1I	hv_baseline;1	hv_baseline
  KEY: TH1I	hw_baseline;1	hw_baseline
  KEY: TH2F	h_raw;1	h_raw
  KEY: TH2F	hu_raw;1	hu_raw
  KEY: TH2F	hv_raw;1	hv_raw
  KEY: TH2F	hw_raw;1	hw_raw


In [4]:
histname = "h_orig"
hist = rfile.Get(histname)

In [5]:
xbins = hist.GetNbinsX()
ybins = hist.GetNbinsY()

In [6]:
## use the rainbow palette which has better visual
ROOT.gStyle.SetPalette(ROOT.kRainBow)
canvas = ROOT.TCanvas()
canvas.cd()
hist.SetStats(0)
hist.SetTitle("Proto-DUNE Raw Data")
hist.GetXaxis().SetTitle("Channel Numbers")
hist.GetYaxis().SetTitle("NTicks")
hist.SetMaximum(50)
hist.SetMinimum(-50)
hist.Draw("colz")
canvas.SaveAs(histname+".png")

Info in <TCanvas::Print>: png file h_orig.png has been created


In [7]:
print(xbins,ybins)
#also do the total projection of Y
totY = hist.ProjectionY()
totY.SetTitle("Total ADC")
totY.GetXaxis().SetTitle("Number of Ticks")
totY.GetYaxis().SetTitle("Number of Counts")
totY.SetStats(0)
canvas.cd()
totY.Draw("hist")
canvas.SaveAs(histname+"_totY.png")

10240 5859


Info in <TCanvas::Print>: png file h_orig_totY.png has been created


## Xbins -> Channel numbers (Each apa has 2560 channels). This file for 4 APAS
## YBins --> Ticks

In [8]:
ofilename = rfilename.replace(".root","_1D.root")
ofile = ROOT.TFile(ofilename,"RECREATE")
ofile.cd()
#now create an output file for each 
for i in range(0,xbins): # for each channels
    xlow = i
    xhigh = i+1
    temp_name = "waveform_"+str(xlow)
    temp_hist = hist.ProjectionY(temp_name,xlow, xhigh,"")
    temp_hist.Write()
        
ofile.Write()

103903057

In [9]:
ofile.Close()

## now write the 1-D waveforms in the HDF5 File as well..

In [10]:
h5filename = ofilename.replace(".root",".h5")
hfile = h5py.File(h5filename,'w')

In [11]:
ofile = ROOT.TFile(ofilename,"READ")
for i in range(0,xbins):
    dsetname = "waveform_"+str(i)
    temp_hist = ofile.Get(dsetname)
    bin_content = np.array([temp_hist.GetBinContent(i) for i in range(0,temp_hist.GetNbinsX()+1)])
    hfile.create_dataset(dsetname,data=bin_content)

ofile.Close()
hfile.close()